# CADS API Python client Tests

In [ ]:
import os
import xarray as xr

import cads_api_client

In [ ]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.copernicus-climate.eu/api")
api_url

## Client instantiation

The client exposes the APIs for:
- Catalogue exploration (see **Section 1**)
- Data retrieval (see **Section 2**)

In [ ]:
client = cads_api_client.ApiClient(api_url)
client

## 1. Catalogue Exploration


### 1.1 Collections

**Objective**: Verify the access to the list of **collections** and its description.

_**Expected result**: Correct instantiation of the collections object._

*If `collections.response` returns status code 200, the request has been successful and the test can proceed.*

In [ ]:
collections = client.collections()
collections.response

_**Expected result**: List of all available collections._

In [ ]:
collections.collection_ids()

### 1.2 Collection

**Objective**: Verify the access to the list of **collection** and its description.

_**Expected result**: Correct instantiation of the collection object._

*If `collection.response` returns status code 200, the request has been successful and the test can proceed.*

In [ ]:
collection = client.collection("reanalysis-era5-pressure-levels")
collection

_**Expected result**: JSON of the collection response describing the collection._

In [ ]:
collection.json

## 2. Data Retrieval

The retrieval can be done using a high-level function (`client.retrieve`) that performs the submission, monitoring and download (see **Section 2.1**).

Alternatively, it can be done using the low-level API (see **Section 3**): 
- **Section 3.1**: `client.submit` for submitting the request
- **Section 3.2**: `remote.status` for monitoring the request
- **Section 3.3**: `remote.download` for downloading the result


### 2.1  Retrieval: Small data request

**Objective**: Verify the capabilities of the client to retrieve data.


The **retrieve** function is blocking. It returns after completing the following steps: 
- Submit the request
- Wait until the request is complete
- Download data


_**Expected result**: The client submits the request; when the process is complete, the client downloads the file._

In [ ]:
output_path = client.retrieve(
    "reanalysis-era5-pressure-levels",
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="1971", 
    month="01", 
    day="25", 
    time="06:00",
    target="test01.grib",
)

output_path

In [ ]:
ls -l $output_path

_**Expected output**: Data consistent with the request:_
- _One variable: t (temperature)_
- _Two dimensions: latitude and longitude_
- _The time coordinate has length 1_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 3 Data Retrieval: advanced API


### 3.1 Request Submission: Big data request

**Objective:** Verify the capability of the client to submit a request.



`client.submit` is a non-blocking function. It returns a remote object that allows to monitor the request status.


_**Expected result**: The client submits the request and returns a remote object that allows to monitor the process._

In [ ]:
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="1971", 
    month=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    day=[
        '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
        '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
        '25', '26', '27', '28', '29', '30', '31',
    ],
    time="06:00",
    target="test02.grib",
)
remote.request_uid

### 3.2 Request Monitoring

**Objectives**: Verify the client capability of monitoring requests.

`remote.status` allows to monitor the status of the process.

_**Expected result**: The updated status of the request is returned (failed, successful, or running)._

In [ ]:
remote.status

`client.get_requests` returns the list of requests submitted.

_**Expected result**: IDs of the submitted requests._

In [ ]:
requests = client.get_requests()
requests.job_ids()

_**Expected result**: The remote ID is in the list of the submitted requests._

In [ ]:
remote.request_uid in requests.job_ids()

### 3.3 Data download

**Objectives**: Verify the client capability of downloading data.

The `download` function is blocking. It returns after completing the following steps: 
- Wait until the request is complete
- Download the data requested

_**Expected result**: The file is downloaded and saved in `$output_path`._

In [ ]:
output_path = remote.download("test02.grib")
output_path

In [ ]:
ls -l $ouput_path

_**Expected output**: Data consistent with the request:_
- _One variable: t (temperature)_
- _Three dimensions: time (365), latitude (721) and longitude (1440)_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 4. Error Handling

**Objectives**: Verify how errors are handled through error messages and codes.

### 4.1 Wrong URL

_**Expected output**: 404 Client Error: Not Found for url._

In [ ]:
client = cads_api_client.ApiClient(f"{api_url}_1")
client.collections()

### 4.2 Missing collection

_**Expected output**: 404 Client Error: Not Found for url._

In [ ]:
client = cads_api_client.ApiClient(api_url)
client.collection("missing_collection")

### 4.3 Unknown job

_**Expected output**: 404 Client Error: Not Found for url._

In [ ]:
status_info = client.get_request("ffffffff-4455-6677-8899-aabbccddeeff")

### 4.4 Wrong request

**Non blocking request using `collection.submit`.**

_**Expected output**: `collection.submit` does not raise errors._


In [ ]:
client = cads_api_client.ApiClient(api_url)
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib",
    product_type="reanalysis", 
    variable="temperature", 
    pressure_level="1", 
    year="2222", 
    month="01", 
    day="25", 
    time="06:00",
    format="grib", 
)

_**Expected output**: `remote.wait_on_result` raises a `ProcessingFailedError`._

In [ ]:
remote.wait_on_result()

_**Expected output**: The result status code is 400._

In [ ]:
results = remote.make_results()

In [ ]:
results.status_code

_**Expected output**:_
- _`result.json` describes the error in the fields: "type", "title" and "detail"_
- _"detail" contains the error traceback_

In [ ]:
results.json